# GTFS Bus Data Analysis

This notebook analyzes NYC MTA bus GTFS data including:
1. **GTFS Schedule Data** - Static transit schedule from `data/gtfs_b/`
2. **GTFS Realtime Data** - Real-time vehicle positions from parquet files

## Part 1: GTFS Schedule Summary

In [11]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load GTFS schedule files
gtfs_path = 'data/gtfs_b/'

print("Loading GTFS Schedule Data...")
print("="*60)

# Load key GTFS files
calendar = pd.read_csv(gtfs_path + 'calendar.txt')
calendar_dates = pd.read_csv(gtfs_path + 'calendar_dates.txt')
routes = pd.read_csv(gtfs_path + 'routes.txt')
trips = pd.read_csv(gtfs_path + 'trips.txt')
stops = pd.read_csv(gtfs_path + 'stops.txt')
agency = pd.read_csv(gtfs_path + 'agency.txt')

print("✓ Loaded all GTFS files successfully")

Loading GTFS Schedule Data...
✓ Loaded all GTFS files successfully


In [12]:
# Service Date Coverage
print("\n📅 SERVICE DATE COVERAGE")
print("="*60)

# Convert date columns to datetime
calendar['start_date'] = pd.to_datetime(calendar['start_date'], format='%Y%m%d')
calendar['end_date'] = pd.to_datetime(calendar['end_date'], format='%Y%m%d')
calendar_dates['date'] = pd.to_datetime(calendar_dates['date'], format='%Y%m%d')

# Calendar (regular service patterns)
print("\nRegular Service Patterns (calendar.txt):")
print(f"  Service start: {calendar['start_date'].min().strftime('%Y-%m-%d')}")
print(f"  Service end:   {calendar['end_date'].max().strftime('%Y-%m-%d')}")
print(f"  Number of service patterns: {len(calendar)}")
print(f"\n  Service IDs by day of week:")
for idx, row in calendar.iterrows():
    days = []
    if row['monday'] == 1: days.append('Mon')
    if row['tuesday'] == 1: days.append('Tue')
    if row['wednesday'] == 1: days.append('Wed')
    if row['thursday'] == 1: days.append('Thu')
    if row['friday'] == 1: days.append('Fri')
    if row['saturday'] == 1: days.append('Sat')
    if row['sunday'] == 1: days.append('Sun')
    print(f"    {row['service_id']}: {', '.join(days)}")

# Calendar dates (exceptions)
print(f"\nService Exceptions (calendar_dates.txt):")
print(f"  Date range: {calendar_dates['date'].min().strftime('%Y-%m-%d')} to {calendar_dates['date'].max().strftime('%Y-%m-%d')}")
print(f"  Total exception dates: {len(calendar_dates)}")
print(f"  Added service (1): {(calendar_dates['exception_type'] == 1).sum()} dates")
print(f"  Removed service (2): {(calendar_dates['exception_type'] == 2).sum()} dates")

# Overall date coverage
all_dates = pd.concat([
    calendar[['start_date']].rename(columns={'start_date': 'date'}),
    calendar[['end_date']].rename(columns={'end_date': 'date'}),
    calendar_dates[['date']]
])
print(f"\n📊 OVERALL DATE RANGE: {all_dates['date'].min().strftime('%Y-%m-%d')} to {all_dates['date'].max().strftime('%Y-%m-%d')}")


📅 SERVICE DATE COVERAGE

Regular Service Patterns (calendar.txt):
  Service start: 2025-01-29
  Service end:   2025-06-28
  Number of service patterns: 29

  Service IDs by day of week:
    EN_B5-Sunday: Sun
    EN_B5-Weekday: Mon, Tue, Wed, Thu, Fri
    EN_B5-Weekday-SDon: Mon, Tue, Wed, Thu, Fri
    EN_B5-Saturday: Sat
    FB_B5-Sunday: Sun
    FB_B5-Weekday: Mon, Tue, Wed, Thu, Fri
    FB_B5-Weekday-SDon: Mon, Tue, Wed, Thu, Fri
    FB_B5-Saturday: Sat
    FP_B5-Sunday: Sun
    FP_B5-Weekday: Mon, Tue, Wed, Thu, Fri
    FP_B5-Weekday-SDon: Mon, Tue, Wed, Thu, Fri
    FP_B5-Saturday: Sat
    GA_B5-Sunday: Sun
    GA_B5-Weekday: Mon, Tue, Wed, Thu, Fri
    GA_B5-Weekday-SDon: Mon, Tue, Wed, Thu, Fri
    GA_B5-Saturday: Sat
    GA_B5-Saturday-BM: Fri
    JG_B5-Sunday: Sun
    JG_B5-Sunday-BM: Sat
    JG_B5-Weekday: Mon, Tue, Wed, Thu, Fri
    JG_B5-Weekday-BM: Mon, Tue, Wed, Thu, Sun
    JG_B5-Weekday-SDon: Mon, Tue, Wed, Thu, Fri
    JG_B5-Weekday-SDon-BM: Mon, Tue, Wed, Thu, Sun
   

In [13]:
# Routes Summary
print("\n🚌 ROUTES SUMMARY")
print("="*60)

print(f"\nTotal routes: {len(routes)}")
print(f"Route types:")
route_types = routes['route_type'].value_counts()
route_type_names = {3: 'Bus', 0: 'Light Rail', 1: 'Subway/Metro', 2: 'Rail'}
for route_type, count in route_types.items():
    type_name = route_type_names.get(route_type, f'Type {route_type}')
    print(f"  {type_name}: {count} routes")

print(f"\nSample routes:")
print(routes[['route_id', 'route_short_name', 'route_long_name', 'route_type']].head(10).to_string(index=False))


🚌 ROUTES SUMMARY

Total routes: 314
Route types:
  Bus: 274 routes
  Type 711: 40 routes

Sample routes:
route_id route_short_name                  route_long_name  route_type
      B1               B1      Bay Ridge - Manhattan Beach           3
     B11              B11            Sunset Park - Midwood           3
     B12              B12 Lefferts Gardens - East New York           3
     B13              B13  Spring Creek - Wyckoff Hospital           3
     B14              B14     Spring Creek - Crown Heights           3
     B15              B15 Bedford Stuyvesant - JFK Airport           3
     B16              B16     Bay Ridge - Lefferts Gardens           3
     B17              B17         Canarsie - Crown Heights           3
      B2               B2  Kings Hwy Station - Kings Plaza           3
     B20              B20         Ridgewood - Spring Creek           3


In [14]:
# Trips Summary
print("\n🚏 TRIPS SUMMARY")
print("="*60)

print(f"\nTotal scheduled trips: {len(trips):,}")
print(f"Unique routes with trips: {trips['route_id'].nunique()}")
print(f"Unique service IDs: {trips['service_id'].nunique()}")

# Trips per route
trips_per_route = trips.groupby('route_id').size().sort_values(ascending=False)
print(f"\nTop 10 routes by number of scheduled trips:")
for route_id, count in trips_per_route.head(10).items():
    route_name = routes[routes['route_id'] == route_id]['route_short_name'].values[0] if len(routes[routes['route_id'] == route_id]) > 0 else route_id
    print(f"  {route_name:6s} ({route_id}): {count:,} trips")

# Direction distribution
if 'direction_id' in trips.columns:
    print(f"\nTrips by direction:")
    print(f"  Direction 0: {(trips['direction_id'] == 0).sum():,}")
    print(f"  Direction 1: {(trips['direction_id'] == 1).sum():,}")


🚏 TRIPS SUMMARY

Total scheduled trips: 45,866
Unique routes with trips: 65
Unique service IDs: 29

Top 10 routes by number of scheduled trips:
  B6     (B6): 1,806 trips
  B41    (B41): 1,737 trips
  Q58    (Q58): 1,688 trips
  B35    (B35): 1,450 trips
  B38    (B38): 1,220 trips
  B46    (B46): 1,188 trips
  B15    (B15): 1,165 trips
  B46-SBS (B46+): 1,117 trips
  B17    (B17): 1,088 trips
  B12    (B12): 1,042 trips

Trips by direction:
  Direction 0: 22,715
  Direction 1: 23,151


In [15]:
# Stops Summary
print("\n📍 STOPS SUMMARY")
print("="*60)

print(f"\nTotal stops: {len(stops):,}")

# Geographic bounds
print(f"\nGeographic coverage:")
print(f"  Latitude range:  {stops['stop_lat'].min():.6f} to {stops['stop_lat'].max():.6f}")
print(f"  Longitude range: {stops['stop_lon'].min():.6f} to {stops['stop_lon'].max():.6f}")

# Location types if available
if 'location_type' in stops.columns:
    print(f"\nLocation types:")
    location_types = stops['location_type'].value_counts()
    location_type_names = {0: 'Stop/Platform', 1: 'Station', 2: 'Entrance/Exit', 3: 'Generic Node', 4: 'Boarding Area'}
    for loc_type, count in location_types.items():
        type_name = location_type_names.get(loc_type, f'Type {loc_type}')
        print(f"  {type_name}: {count:,}")

print(f"\nSample stops:")
print(stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']].head(5).to_string(index=False))


📍 STOPS SUMMARY

Total stops: 4,619

Geographic coverage:
  Latitude range:  40.572635 to 40.762524
  Longitude range: -74.040876 to -73.789580

Location types:
  Stop/Platform: 4,619

Sample stops:
 stop_id                  stop_name  stop_lat   stop_lon
  300000 ORIENTAL BLVD/MACKENZIE ST 40.578350 -73.940029
  300002   ORIENTAL BLVD/JAFFRAY ST 40.578066 -73.943029
  300003  ORIENTAL BLVD/HASTINGS ST 40.577909 -73.944643
  300004  ORIENTAL BLVD/FALMOUTH ST 40.577718 -73.946200
  300006     ORIENTAL BLVD/DOVER ST 40.577353 -73.949552


In [16]:
# Overall GTFS Schedule Summary
print("\n" + "="*60)
print("📊 GTFS SCHEDULE SUMMARY")
print("="*60)

print(f"\nAgency: {agency['agency_name'].values[0] if len(agency) > 0 else 'N/A'}")
print(f"\nService Period: {calendar['start_date'].min().strftime('%Y-%m-%d')} to {calendar['end_date'].max().strftime('%Y-%m-%d')}")
print(f"Total Routes: {len(routes):,}")
print(f"Total Trips: {len(trips):,}")
print(f"Total Stops: {len(stops):,}")
print(f"Average Trips per Route: {len(trips) / len(routes):.1f}")

print("\n" + "="*60)
print("Ready to analyze realtime data!")


📊 GTFS SCHEDULE SUMMARY

Agency: MTA New York City Transit

Service Period: 2025-01-29 to 2025-06-28
Total Routes: 314
Total Trips: 45,866
Total Stops: 4,619
Average Trips per Route: 146.1

Ready to analyze realtime data!


---
## Part 2: GTFS Realtime Data Analysis

In [ ]:
# Load realtime vehicle position data
print("Loading Realtime Data...")
print("="*60)

df = pd.read_parquet('data/COMPACTED_nyct_mta_bus_gtfsrt_2025-06-02.parquet')

print(f"✓ Loaded {len(df):,} realtime vehicle position records")
print(f"  Date: June 2, 2025")
print(f"  Time range: {df['vehicle.timestamp'].min()} to {df['vehicle.timestamp'].max()}")
print(f"  Total routes: {df['vehicle.trip.route_id'].nunique()}")
print(f"  Total trips: {df['vehicle.trip.trip_id'].nunique()}")

In [ ]:
# Filter for B46 route
print("\n🚌 Filtering for B46 Route")
print("="*60)

df_b46 = df[df['vehicle.trip.route_id'] == 'B46'].copy()

print(f"\nB46 Route Statistics:")
print(f"  Total records: {len(df_b46):,}")
print(f"  Unique trips: {df_b46['vehicle.trip.trip_id'].nunique()}")
print(f"  Unique vehicles: {df_b46['vehicle.vehicle.id'].nunique()}")
print(f"  Time range: {df_b46['vehicle.timestamp'].min()} to {df_b46['vehicle.timestamp'].max()}")

# Check for records without trip_id
missing_trip_id = df_b46['vehicle.trip.trip_id'].isna().sum()
if missing_trip_id > 0:
    print(f"  ⚠️  Records without trip_id: {missing_trip_id}")
    df_b46 = df_b46.dropna(subset=['vehicle.trip.trip_id'])
    print(f"  Continuing with {len(df_b46):,} records that have trip_ids")

df_b46.head()

In [ ]:
# Create trip-level summary with observation counts
print("\n📊 B46 Trip-Level Summary")
print("="*60)

# Count observations per trip
b46_trip_summary = df_b46.groupby('vehicle.trip.trip_id').agg({
    'vehicle.trip.trip_id': 'count',
    'vehicle.trip.direction_id': 'first',
    'vehicle.timestamp': ['min', 'max']
}).reset_index()

# Flatten column names
b46_trip_summary.columns = ['trip_id', 'num_observations', 'direction_id', 'first_timestamp', 'last_timestamp']

# Calculate duration
b46_trip_summary['duration_minutes'] = (
    b46_trip_summary['last_timestamp'] - b46_trip_summary['first_timestamp']
).dt.total_seconds() / 60

print(f"\nCreated summary for {len(b46_trip_summary)} unique B46 trips")
print(f"Observation count range: {b46_trip_summary['num_observations'].min()} to {b46_trip_summary['num_observations'].max()}")
print(f"\nSample:")
b46_trip_summary.head(10)

In [ ]:
# Check if trips exist in GTFS schedule
print("\n🔍 Checking Trips Against GTFS Schedule")
print("="*60)

# Filter GTFS trips for B46 route
gtfs_b46_trips = trips[trips['route_id'] == 'B46'].copy()

print(f"\nGTFS Schedule for B46:")
print(f"  Scheduled trips in GTFS: {len(gtfs_b46_trips)}")
print(f"  Observed trips in realtime: {len(b46_trip_summary)}")

# Check which observed trips are in the schedule
b46_trip_summary['in_schedule'] = b46_trip_summary['trip_id'].isin(gtfs_b46_trips['trip_id'])

in_schedule_count = b46_trip_summary['in_schedule'].sum()
not_in_schedule_count = (~b46_trip_summary['in_schedule']).sum()

print(f"\nMatch Results:")
print(f"  ✓ Trips found in schedule: {in_schedule_count}")
print(f"  ✗ Trips NOT in schedule: {not_in_schedule_count}")
print(f"  Match rate: {in_schedule_count / len(b46_trip_summary) * 100:.1f}%")

# Show some examples of trips not in schedule
if not_in_schedule_count > 0:
    print(f"\nSample trips NOT found in schedule:")
    print(b46_trip_summary[~b46_trip_summary['in_schedule']][['trip_id', 'num_observations', 'direction_id']].head())

In [ ]:
# Get first stop information from schedule
print("\n🚏 Getting First Stop Information from Schedule")
print("="*60)

print("Loading stop_times.txt (this may take a moment)...")
stop_times = pd.read_csv(gtfs_path + 'stop_times.txt')

print(f"✓ Loaded {len(stop_times):,} stop time records")

# Get first stop for each B46 trip (stop_sequence = 1 or minimum)
b46_trip_ids = gtfs_b46_trips['trip_id'].tolist()
stop_times_b46 = stop_times[stop_times['trip_id'].isin(b46_trip_ids)].copy()

print(f"  Stop times for B46 trips: {len(stop_times_b46):,}")

# Get the first stop for each trip (lowest stop_sequence)
first_stops = stop_times_b46.loc[stop_times_b46.groupby('trip_id')['stop_sequence'].idxmin()][
    ['trip_id', 'arrival_time', 'stop_id', 'stop_sequence']
].copy()

print(f"  First stops identified: {len(first_stops)}")

# Join with stops to get stop names
first_stops = first_stops.merge(
    stops[['stop_id', 'stop_name']], 
    on='stop_id', 
    how='left'
)

print(f"\nSample first stops:")
print(first_stops.head())

In [ ]:
# Create final trip analysis table
print("\n📋 Final B46 Trip Analysis Table")
print("="*60)

# Merge trip summary with first stop information
b46_final = b46_trip_summary.merge(
    first_stops[['trip_id', 'arrival_time', 'stop_name', 'stop_id']], 
    on='trip_id', 
    how='left'
)

# Rename columns for clarity
b46_final = b46_final.rename(columns={
    'arrival_time': 'scheduled_first_stop_time',
    'stop_name': 'scheduled_first_stop_name',
    'stop_id': 'scheduled_first_stop_id'
})

# Sort by in_schedule (True first) and then by num_observations (descending)
b46_final = b46_final.sort_values(['in_schedule', 'num_observations'], ascending=[False, False])

print(f"\nFinal table created with {len(b46_final)} trips")
print(f"\nColumns:")
for col in b46_final.columns:
    print(f"  - {col}")

print(f"\n{'='*60}")
print("Summary Statistics:")
print(f"  Trips in schedule: {b46_final['in_schedule'].sum()}")
print(f"  Trips not in schedule: {(~b46_final['in_schedule']).sum()}")
print(f"  Average observations per trip: {b46_final['num_observations'].mean():.1f}")
print(f"  Median observations per trip: {b46_final['num_observations'].median():.1f}")

# Display the final table
b46_final

In [ ]:
# Additional insights and filtered views
print("\n🔎 Additional Insights")
print("="*60)

# Show trips in schedule vs not
print("\n1. TRIPS IN SCHEDULE:")
print("-" * 60)
in_sched = b46_final[b46_final['in_schedule']].copy()
print(f"   Count: {len(in_sched)}")
if len(in_sched) > 0:
    print(f"   Observations: {in_sched['num_observations'].sum():,} total")
    print(f"   Average duration: {in_sched['duration_minutes'].mean():.1f} minutes")
    print(f"\n   Sample (top 10 by observations):")
    display(in_sched[['trip_id', 'num_observations', 'scheduled_first_stop_time', 
                       'scheduled_first_stop_name', 'duration_minutes']].head(10))

print("\n2. TRIPS NOT IN SCHEDULE:")
print("-" * 60)
not_in_sched = b46_final[~b46_final['in_schedule']].copy()
print(f"   Count: {len(not_in_sched)}")
if len(not_in_sched) > 0:
    print(f"   Observations: {not_in_sched['num_observations'].sum():,} total")
    print(f"   Average duration: {not_in_sched['duration_minutes'].mean():.1f} minutes")
    print(f"\n   Sample (top 10 by observations):")
    display(not_in_sched[['trip_id', 'num_observations', 'direction_id', 
                           'first_timestamp', 'duration_minutes']].head(10))

# Direction breakdown
print("\n3. BREAKDOWN BY DIRECTION:")
print("-" * 60)
direction_stats = b46_final.groupby('direction_id').agg({
    'trip_id': 'count',
    'num_observations': 'sum',
    'in_schedule': 'sum'
}).rename(columns={'trip_id': 'trip_count', 'in_schedule': 'scheduled_trip_count'})
print(direction_stats)

In [ ]:
# Create a clean export version of the table
print("\n💾 Export-Ready Table")
print("="*60)

# Select and reorder columns for export
export_columns = [
    'trip_id',
    'num_observations', 
    'in_schedule',
    'scheduled_first_stop_time',
    'scheduled_first_stop_name',
    'direction_id',
    'first_timestamp',
    'last_timestamp',
    'duration_minutes'
]

b46_export = b46_final[export_columns].copy()

# Format the table for better readability
b46_export['in_schedule'] = b46_export['in_schedule'].map({True: 'Yes', False: 'No'})
b46_export['duration_minutes'] = b46_export['duration_minutes'].round(1)

print(f"\nExport table ready with {len(b46_export)} rows and {len(b46_export.columns)} columns")
print(f"\nYou can export this to CSV with:")
print("  b46_export.to_csv('b46_trip_analysis.csv', index=False)")

print(f"\nFull table preview (showing all {len(b46_export)} rows):")
b46_export